# Prétraitement des Données Moléculaires

Ce notebook contient les fonctions nécessaires pour prétraiter les données moléculaires brutes (coordonnées atomiques, etc.) afin de les transformer en un format adapté à l'entraînement d'un modèle d'apprentissage automatique. Les étapes principales incluent la création de champs de points autour des atomes, le calcul de potentiels et la structuration des données pour chaque molécule.

## Importation des Bibliothèques

Importation des bibliothèques Python nécessaires telles que :  
- **collections** : pour créer un dictionnaire de listes, 
- **numpy** : pour les calculs numériques,
- **scipy** : pour la distance euclidienne, 
- **basis_set_exchange** : pour la gestion des bases de données de fonctions d'onde
- **mendeleev** : pour la gestion des éléments chimiques

In [1]:
import argparse
from collections import defaultdict
import os
import pickle

import numpy as np

from scipy import spatial
import plotly.graph_objects as go

import basis_set_exchange as bse
from mendeleev import element

## Dictionnaire des Numéros Atomiques

Création d'un dictionnaire qui mappe les symboles des éléments chimiques (ex: 'H', 'C', 'O') à leurs numéros atomiques correspondants (ex: 1, 6, 8). Ceci est utile pour récupérer des informations spécifiques à chaque atome.

In [2]:
# Dictionnaire de nombres atomiques -> symboles

periodic_table_atoms = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
                'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca',
                'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr',
                'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
                'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
                'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
                'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
                'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm',
                'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds',
                'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

# Création du dictionnaire associant symbole atomique à numéro atomique
atomic_numbers_dict = {periodic_table_atoms[i-1] : i for i in range(1, len(periodic_table_atoms)+1)}
print(f"Nombre d'éléments dans le dictionnaire : {len(atomic_numbers_dict)}")

Nombre d'éléments dans le dictionnaire : 118


## Création d'une Sphère de Points

Cette fonction génère un ensemble de points répartis uniformément à l'intérieur d'une sphère d'un rayon donné. Ces points seront utilisés pour représenter l'espace autour de chaque atome dans une molécule.

In [3]:
def create_sphere(radius, grid_interval):
    """ Crée un ensemble de points formant une sphère pour modéliser l'espace autour de chaque atome.
    
    Args:
        radius (float): Le rayon de la sphère.
        grid_interval (float): L'espacement entre les points de la grille.
        
    Returns:
        np.array: Un tableau NumPy contenant les coordonnées [x, y, z] des points dans la sphère.
    """
    # Génère une grille de points cubique
    xyz = np.arange(-radius, radius + 1e-3, grid_interval)
    # Sélectionne les points à l'intérieur de la sphère (excluant le centre [0,0,0])
    sphere = [[x, y, z] for x in xyz for y in xyz for z in xyz
              if (x**2 + y**2 + z**2 <= radius**2) and not np.allclose([x, y, z], [0, 0, 0])]
    return np.array(sphere)

## Création du Champ Moléculaire

Cette fonction prend la sphère de points générée précédemment et la positionne autour de chaque atome de la molécule. Le résultat est un "champ" de points représentant l'espace pertinent autour de la molécule entière.

In [4]:
def create_field(sphere, coords):
    """ Crée le champ de points pour la molécule en superposant la sphère sur chaque coordonnée atomique.
    
    Args:
        sphere (np.array): Le tableau des points de la sphère généré par `create_sphere`.
        coords (np.array): Les coordonnées [x, y, z] des atomes de la molécule.
        
    Returns:
        np.array: Un tableau NumPy contenant les coordonnées de tous les points du champ moléculaire.
    """
    # Pour chaque coordonnée atomique, ajoute les coordonnées relatives de la sphère
    field = [f + coord for coord in coords for f in sphere]
    return np.array(field)

## Création de la Matrice des Distances

Calcule la matrice des distances euclidiennes entre deux ensembles de coordonnées. Typiquement utilisé pour calculer les distances entre les points du champ moléculaire et les positions des atomes (ou des orbitales).

In [5]:
def create_distancematrix(coords1, coords2):
    """ Crée la matrice des distances entre deux ensembles de coordonnées.
    
    Args:
        coords1 (np.array): Premier ensemble de coordonnées (ex: points du champ).
        coords2 (np.array): Deuxième ensemble de coordonnées (ex: positions atomiques).
        
    Returns:
        np.array: Matrice où l'élément (i, j) est la distance entre le i-ème point de coords1 et le j-ème point de coords2.
                Les distances nulles sont remplacées par 1e6 pour éviter les problèmes avec les calculs gaussiens.
    """
    # Calcule la matrice des distances euclidiennes
    distance_matrix = spatial.distance_matrix(coords1, coords2)
    # Remplace les distances nulles (points superposés) par une grande valeur
    return np.where(distance_matrix == 0.0, 1e6, distance_matrix)

## Encodage des Orbitales Atomiques

Transforme les descriptions textuelles des orbitales atomiques (ex: 'H1s0', 'C2p1') en indices numériques uniques à l'aide d'un dictionnaire. Cela permet de représenter les types d'orbitales de manière catégorielle pour le modèle.

In [6]:
def create_orbitals(orbitals, orbital_dict):
    """ Transforme les noms d'orbitales en indices numériques en utilisant un dictionnaire.
    
    Args:
        orbitals (list): Liste des noms d'orbitales (ex: ['H1s0', 'C1s0', ...]).
        orbital_dict (defaultdict): Dictionnaire mappant les noms d'orbitales à des indices entiers.
                                     Utilise `defaultdict` pour assigner automatiquement un nouvel indice si une orbitale n'est pas encore présente.
                                     
    Returns:
        np.array: Tableau NumPy contenant les indices numériques des orbitales.
    """
    # Mappe chaque nom d'orbitale à son indice correspondant dans le dictionnaire
    orbitals_indices = [orbital_dict[o] for o in orbitals]
    return np.array(orbitals_indices)

## Calcul du Potentiel Externe

Calcule un potentiel externe basé sur une somme pondérée de fonctions gaussiennes centrées sur les atomes. Ce potentiel est inspiré par des approches de type Kohn-Sham en théorie de la fonctionnelle de la densité (DFT) et sert de caractéristique d'entrée pour le modèle.

- **Ref article** : Brockherde et al., 2017, Bypassing the Kohn-Sham equations with machine learning

In [7]:
def create_potential(distance_matrix, atomic_numbers):
    """ Crée le potentiel gaussien externe basé sur les distances aux noyaux atomiques.
    
    Args:
        distance_matrix (np.array): Matrice des distances entre les points du champ et les atomes.
        atomic_numbers (np.array): Tableau des numéros atomiques des atomes de la molécule.
        
    Returns:
        np.array: Tableau représentant le potentiel en chaque point du champ.
    """
    Gaussians = np.exp(-distance_matrix**2)
    # Calcule le potentiel comme une somme pondérée par les numéros atomiques (charge nucléaire)
    # Le signe négatif indique un potentiel attractif.
    return -1 * np.matmul(Gaussians, atomic_numbers)

## Création du Jeu de Données Prétraité

Fonction principale qui orchestre le processus de prétraitement. Elle lit les données brutes d'un fichier texte, applique les fonctions définies précédemment (création de sphère, champ, calcul des distances, potentiel, encodage des orbitales) pour chaque molécule, et sauvegarde les données prétraitées dans des fichiers NumPy individuels (`.npy`).

In [ ]:
def create_dataset(dir_dataset, filename, basis_set, radius, grid_interval, orbital_dict, property=True):
    """ Fonction principale pour prétraiter un ensemble de données moléculaires.
    
    Args:
        dir_dataset (str): Répertoire contenant le fichier de données brutes et où sauvegarder les données prétraitées.
        filename (str): Nom du fichier de données brutes (sans extension).
        basis_set (str): Nom du basis set à utiliser (ex: 'def2-SVP').
        radius (float): Rayon de la sphère autour de chaque atome.
        grid_interval (float): Espacement de la grille pour la sphère.
        orbital_dict (defaultdict): Dictionnaire pour encoder les orbitales.
        property (bool): Indique si le fichier de données contient des valeurs de propriétés à extraire.
    """
    
    # Définit le répertoire de sortie pour les données prétraitées
    if property:
        dir_preprocessed = (dir_dataset + filename + "/" + filename + '_' + basis_set + "_sphere_" + str(radius) + "_" + str(grid_interval) + 'grid/')
    else:  
        dir_preprocessed = dir_dataset + filename + '/'
    
    # Crée le répertoire de sortie s'il n'existe pas
    os.makedirs(dir_preprocessed, exist_ok=True)
    
    # Récupère les métadonnées du basis set depuis Basis Set Exchange
    try:
        metadata = bse.filter_basis_sets()[basis_set.lower()]
        latest_version = metadata.get("latest_version")
        last_element_in_basis_set = len(metadata['versions'][str(latest_version)]['elements'])
    except KeyError:
         raise ValueError(f"Basis set '{basis_set}' non trouvé dans Basis Set Exchange.")
    
    # Vérifie que le basis set utilise des orbitales de type Gaussien (GTO)
    if not "gto" in metadata['function_types']:
        raise ValueError("Le basis set fourni n'utilise pas de fonctions GTO.")

    # Télécharge les informations détaillées du basis set pour les éléments concernés
    basis_set_exchange = bse.get_basis(basis_set, elements=[i for i in range(1, last_element_in_basis_set + 1)])
    
    if not basis_set_exchange:
        raise ValueError(f"Invalid basis set: {basis_set}")
    
    # Crée la sphère de points une seule fois
    sphere = create_sphere(radius, grid_interval)
    
    # On récupère les fichiers avec les coordonnées
    with open(dir_dataset + filename + "/" + filename + "_m3d" + '.txt', 'r') as f:
        dataset = f.read().strip().split('\n\n')
    
    # Traite chaque molécule dans le jeu de données
    for n, data in enumerate(dataset):
        data = data.strip().split('\n')
        
        # Extrait l'identifiant de la molécule
        idx = data[0]
        print(f"Traitement de la molécule : {idx}") # Ajout pour suivi
        
        # Extrait les coordonnées atomiques et éventuellement les propriétés
        if property:
            if len(data) < 3:
                print(f"Attention : Données incomplètes pour {idx}. Skipping.")
                continue
            atom_xyzs = data[1:-1]
            property_values_str = data[-1].strip().split()
            property_values = np.array([[float(p) for p in property_values_str]])
            
        else:
            if len(data) < 2:
                print(f"Attention : Données incomplètes pour {idx}. Skipping.")
                continue
            atom_xyzs = data[1:]
            
        # Initialise les listes pour stocker les informations de la molécule
        atoms = []
        atomic_numbers = []
        N_electrons = 0
        atomic_coords = []
        atomic_orbitals = []
        orbital_coords = [] # Coordonnées associées à chaque orbitale (identiques aux coords atomiques)
        quantum_numbers = [] # Nombre quantique principal associé à chaque orbitale
        
        # Traite chaque ligne atome/coordonnée
        for atom_xyz in atom_xyzs:
            atom, x, y, z = atom_xyz.split()
            atoms.append(atom)
            
            atomic_number = atomic_numbers_dict[atom]
            atomic_numbers.append([atomic_number])
            
            N_electrons += atomic_number
            
            xyz = [float(v) for v in [x, y, z]]
            atomic_coords.append(xyz)
            
            electronic_configuration = element(atom).ec.conf  # on recupere la configuration électronique de l'atome actuel
            
            aqs = [] # Liste pour stocker (nom_orbitale, nombre_quantique_principal)
            
            number_of_functions_primitive = 0
            try:   # si un element n'est pas trouvé dans le basis set, on l'ignore
                electron_shell = basis_set_exchange["elements"][str(atomic_number)]['electron_shells']
            except KeyError:
                print(f"Warning: Numéro atomique de l'élément {atomic_number} n'est pas trouvé dans le basis set. Skipping element {atom}.")
                continue
            
            for atomic_basis_function in  electron_shell:
                
                i = electron_shell.index(atomic_basis_function)
                    
                if i < len(electron_shell) - 1:
                    if electron_shell[i]["angular_momentum"] != electron_shell[i-1]["angular_momentum"]:
                        number_of_functions_primitive = 0
                
                if atomic_basis_function["angular_momentum"] == [0]:
                    for orbital in electronic_configuration:
                        if orbital[1] == "s":
                            for i in range(len(atomic_basis_function["exponents"])):  # on ajoute les orbitales s
                                aqs.append((atom + str(orbital[0]) + orbital[1] + str(number_of_functions_primitive + i), orbital[0]))  # orbital[0] = nombre quantique principal
                    number_of_functions_primitive += len(atomic_basis_function["exponents"])
                
                elif atomic_basis_function["angular_momentum"] == [1]:
                    for orbital in electronic_configuration:
                        if orbital[1] == "p":
                            for i in range(len(atomic_basis_function["exponents"])):  # on ajoute les orbitales p
                                aqs.append((atom + str(orbital[0]) + orbital[1] + str(number_of_functions_primitive + i), orbital[0]))  # orbital[0] = nombre quantique principal
                    number_of_functions_primitive += len(atomic_basis_function["exponents"])
                
                elif atomic_basis_function["angular_momentum"] == [2]:
                    for orbital in electronic_configuration:
                        if orbital[1] == "d":
                            for i in range(len(atomic_basis_function["exponents"])):  # on ajoute les orbitales d
                                aqs.append((atom + str(orbital[0]) + orbital[1] + str(number_of_functions_primitive + i), orbital[0]))  # orbital[0] = nombre quantique principal
                    number_of_functions_primitive += len(atomic_basis_function["exponents"])
                
                elif atomic_basis_function["angular_momentum"] == [3]:
                    for orbital in electronic_configuration:
                        if orbital[1] == "f":
                            for i in range(len(atomic_basis_function["exponents"])):  # on ajoute les orbitales f
                                aqs.append((atom + str(orbital[0]) + orbital[1] + str(number_of_functions_primitive + i), orbital[0]))  # orbital[0] = nombre quantique principal
                    number_of_functions_primitive += len(atomic_basis_function["exponents"])
            
            # Ajoute les orbitales et leurs informations associées pour cet atome
            for orbital_name, n_quantum in aqs:
                atomic_orbitals.append(orbital_name)
                orbital_coords.append(xyz) # Chaque orbitale est centrée sur l'atome
                quantum_numbers.append(n_quantum)
        
        # Vérifie si des atomes/orbitales valides ont été traités pour cette molécule
        if not atomic_coords or not atomic_orbitals:
             print(f"Attention : Aucune donnée atomique/orbitale valide traitée pour {idx}. Skipping molecule.")
             continue
             
        # Convertit les listes en tableaux NumPy
        atomic_coords = np.array(atomic_coords)
        atomic_orbitals_encoded = create_orbitals(atomic_orbitals, orbital_dict) # Encode les noms d'orbitales en entiers
        orbital_coords = np.array(orbital_coords)
        quantum_numbers = np.array([quantum_numbers]) # Ajoute une dimension pour correspondre au format attendu
        atomic_numbers = np.array(atomic_numbers)
        N_electrons = np.array([[N_electrons]])
        
        # Crée le champ de points autour de la molécule
        field_coords = create_field(sphere, atomic_coords)
        N_field = len(field_coords)
        
        # Calcule la matrice des distances entre le champ et les atomes
        distance_matrix_pot = create_distancematrix(field_coords, atomic_coords)
        # Calcule le potentiel externe
        potential = create_potential(distance_matrix_pot, atomic_numbers)
        
        # Calcule la matrice des distances entre le champ et les centres des orbitales
        distance_matrix_orb = create_distancematrix(field_coords, orbital_coords)
        
        # Structure les données prétraitées pour la sauvegarde
        data_to_save = [idx,
                        atomic_orbitals_encoded.astype(np.int64), # Indices des orbitales
                        distance_matrix_orb.astype(np.float32), # Distances champ <-> orbitales
                        quantum_numbers.astype(np.float32), # Nombres quantiques principaux
                        orbital_coords.astype(np.float32), # Coordonnées des centres des orbitales
                        N_electrons.astype(np.float32), # Nombre total d'électrons
                        N_field, # Nombre de points dans le champ
                       ]
        
        # Ajoute les propriétés et le potentiel si disponibles
        if property:
            data_to_save.append(property_values.astype(np.float32)) 
            data_to_save.append(potential.astype(np.float32)) 
        else:
             # Ajoute des placeholders si property=False pour garder une structure cohérente
             data_to_save.append(np.array([[]], dtype=np.float32)) 
             data_to_save.append(potential.astype(np.float32)) 
             
        # Convertit la liste en un tableau NumPy de type 'object' car les éléments ont des formes différentes
        data_array = np.array(data_to_save, dtype=object)
        
        # Sauvegarde les données prétraitées pour cette molécule
        output_path = os.path.join(dir_preprocessed, idx + ".npy")
        np.save(output_path, data_array)
    
    
    # on affiche les molécules avec plotly
    fig = go.Figure()
        
    fig.add_trace(go.Scatter3d(
    x=field_coords[:, 0],
    y=field_coords[:, 1],
    z=field_coords[:, 2],
    mode='markers',
    marker=dict(size=1, color=potential.ravel(), colorscale='Viridis', showscale=True),
    name='Potential'
    ))

    
    bond_threshold = 2
    
    for i in range(len(atomic_coords)):
        for j in range(i+1, len(atomic_coords)):
            dist = np.linalg.norm(atomic_coords[i] - atomic_coords[j])
            if dist <= bond_threshold:
                fig.add_trace(go.Scatter3d(
                    x=[atomic_coords[i][0], atomic_coords[j][0]],
                    y=[atomic_coords[i][1], atomic_coords[j][1]],
                    z=[atomic_coords[i][2], atomic_coords[j][2]],
                    mode='lines+markers',
                    marker=dict(size=4, color='blue'),
                    line=dict(color='blue', width=2),
                    showlegend=False
                ))
    

        fig.show()
        
    print(f"Prétraitement terminé. Données sauvegardées dans : {dir_preprocessed}")

## Exemple d'Utilisation

Démonstration de l'appel de la fonction `create_dataset` avec des paramètres spécifiques (répertoire, nom de fichier, basis set, paramètres de la sphère). Un dictionnaire `orbital_dict` est initialisé pour mapper les noms d'orbitales uniques rencontrées à des indices entiers. Ce dictionnaire est ensuite sauvegardé (sérialisé) à l'aide de `pickle` pour pouvoir être réutilisé lors de l'évaluation du modèle.

In [ ]:
# --- Paramètres pour l'exemple ---
dir_dataset = "../../datasets/PM/"  
filename = "PM"            # Nom du fichier de données brutes (sans extension)
basis_set = "def2-SVP"     # Basis set à utiliser
radius = 0.75              
grid_interval = 0.3        

# Initialise un dictionnaire qui assignera automatiquement un nouvel indice entier
# à chaque nouvelle clé (nom d'orbitale) rencontrée.
orbital_dict = defaultdict(lambda: len(orbital_dict))

# --- Exécution du prétraitement ---
# Appelle la fonction principale pour traiter le fichier 'demo.txt'
# property=True indique que la dernière ligne de chaque molécule dans le dataset contient des propriétés

print('Training dataset...')
create_dataset(dir_dataset, 'train', basis_set, radius, grid_interval, orbital_dict)
print('-'*50)

print('Validation dataset...')
create_dataset(dir_dataset, 'val',
                basis_set, radius, grid_interval, orbital_dict)
print('-'*50)

print('Test dataset...')
create_dataset(dir_dataset, 'test', basis_set, radius, grid_interval, orbital_dict)
print('-'*50)
    
os.makedirs(dir_dataset + 'orbitaldict_' + filename + "/", exist_ok=True)
    
# --- Sauvegarde du dictionnaire d'orbitales ---
# Construit le chemin du fichier pour sauvegarder le dictionnaire

orbital_dict_path = os.path.join(dir_dataset + 'orbitaldict_' + filename + "/", f'orbitaldict_{basis_set}.pickle')
# Ouvre le fichier en mode écriture binaire ('wb')
with open(orbital_dict_path, 'wb') as f:
    # Sérialise (pickle) le dictionnaire (converti en dict standard) dans le fichier
    pickle.dump(dict(orbital_dict), f)
    
print(f"Dictionnaire des orbitales sauvegardé dans : {orbital_dict_path}")
print(f"Nombre total d'orbitales uniques trouvées : {len(orbital_dict)}")

Training dataset...
Traitement de la molécule : pce_0
Traitement de la molécule : pce_1
